# digit.ipynb — Train CNN on MNIST and Save to Google Drive
This notebook mounts Google Drive, trains a simple CNN on the MNIST dataset, and saves the trained model to Drive.

**Steps:**
1) Mount Drive
2) Install/check TensorFlow
3) Load MNIST
4) Build & train CNN
5) Save model (`.h5`) and label map


In [8]:

# 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Choose drive directory
SAVE_DIR = '/content/drive/MyDrive/mnist_digit_cnn'
import os
os.makedirs(SAVE_DIR, exist_ok=True)
print('Saving to:', SAVE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving to: /content/drive/MyDrive/mnist_digit_cnn


In [9]:

# 2) TensorFlow & essentials
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import os, json

print('TensorFlow version:', tf.__version__)
# Enable GPU memory growth (optional, avoids OOM in some setups)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print('GPU memory growth not set:', e)


TensorFlow version: 2.19.0


In [10]:

# 3) Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

# Add channel dimension
x_train = np.expand_dims(x_train, -1)  # (N, 28, 28, 1)
x_test  = np.expand_dims(x_test, -1)

num_classes = 10
print('Train:', x_train.shape, y_train.shape, ' Test:', x_test.shape, y_test.shape)


Train: (60000, 28, 28, 1) (60000,)  Test: (10000, 28, 28, 1) (10000,)


In [11]:

# 4) Build CNN (simple but effective for MNIST)
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,482 (771.41 KB)

 Trainable params: 197,482 (771.41 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:

# 4b) Train
EPOCHS = 8
BATCH_SIZE = 128

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
]

history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 79s 183ms/step - accuracy: 0.7544 - loss: 0.7255 - val_accuracy: 0.9830 - val_loss: 0.0525
Epoch 2/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 81s 180ms/step - accuracy: 0.9677 - loss: 0.1112 - val_accuracy: 0.9895 - val_loss: 0.0364
Epoch 3/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 83s 182ms/step - accuracy: 0.9773 - loss: 0.0753 - val_accuracy: 0.9895 - val_loss: 0.0360
Epoch 4/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 84s 188ms/step - accuracy: 0.9823 - loss: 0.0592 - val_accuracy: 0.9910 - val_loss: 0.0313
Epoch 5/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 81s 185ms/step - accuracy: 0.9855 - loss: 0.0502 - val_accuracy: 0.9935 - val_loss: 0.0259
Epoch 6/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 80s 181ms/step - accuracy: 0.9859 - loss: 0.0452 - val_accuracy: 0.9928 - val_loss: 0.0262
Epoch 7/8
422/422 ━━━━━━━━━━━━━━━━━━━━ 83s 183ms/step - accuracy: 0.9883 - loss: 0.0372 - val_accuracy: 0.9930 - val_loss: 0.0256


In [13]:

# 4c) Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}')


Test accuracy: 0.9919 | Test loss: 0.0251


In [14]:

# 5) Save model & label map to Drive
model_path_h5 = os.path.join(SAVE_DIR, 'mnist_cnn.h5')
model.save(model_path_h5)
print('Saved model:', model_path_h5)

label_map = {i: str(i) for i in range(10)}
with open(os.path.join(SAVE_DIR, 'label_map.json'), 'w') as f:
    json.dump(label_map, f)
print('Saved label_map.json')


Saved model: /content/drive/MyDrive/mnist_digit_cnn/mnist_cnn.h5
Saved label_map.json
